In [ ]:
import glob
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import tensorflow as tf 
from tqdm import tqdm
from tensorflow.keras import optimizers

import sklearn
import numpy as np
import seaborn as sn
from sklearn.metrics import confusion_matrix

In [ ]:
def get_accuracy(labs, pred_labs):
    tp = 0
    for i in range(len(labs)):
        if labs[i]==pred_labs[i]:
            tp = tp+1
            
    return tp*(100/len(labs))

In [ ]:
path = r'D:\Breast cancer material\MIAS_Preprocess\mias_preprocess_Part1\original\train'
folders=glob.glob(os.path.join(path,'*'))
label=[]
i=0
imgs=[]
for fol in tqdm(folders):
    files=glob.glob(os.path.join(fol,'*'))
    for image in files:
        im=cv2.imread(image)
        im= cv2.resize(im,(160,160))/255
        imgs.append(im)
        label.append(i)
    i=i+1
    
imgs=np.array(imgs)
label=np.array(label)
imgs,label=shuffle(imgs,label)

In [ ]:
labs = np.zeros((label.size, label.max()+1))
labs[np.arange(label.size),label] = 1

In [ ]:
img_val = imgs[100:110, :, :,:]
labs_val = labs[100:110,:]
img_val.shape, labs_val.shape

In [ ]:
b = np.zeros((label.size, label.max()+1))
b[np.arange(label.size),label] = 1

In [ ]:
path = r'D:\Breast cancer material\MIAS_Preprocess\mias_preprocess_Part1\original\test'
folders=glob.glob(os.path.join(path,'*'))
label_test=[]
i=0
imgs_test=[]
for fol in tqdm(folders):
    files=glob.glob(os.path.join(fol,'*'))
    for image in tqdm(files):
        im=cv2.imread(image)
        im= cv2.resize(im,(160,160))/255
        imgs_test.append(im)
        label_test.append(i)
    i=i+1
    
imgs_test=np.array(imgs_test)
label_test=np.array(label_test)
imgs_test,label_test=shuffle(imgs_test,label_test)

In [ ]:
b = np.zeros((label.size, label.max()+1))
b[np.arange(label.size),label] = 1
b[100]

In [ ]:
model = tf.keras.applications.Xception(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=(160,160, 3),
    pooling=None,
    classes=3
#     classifier_activation="softmax"
)
model.summary()

In [ ]:
from tensorflow.keras import optimizers
#optimizer=optimizers.SGD(lr=0.01, momentum=0.9)
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-3),
#               optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
batch_size = 9

In [ ]:
import tensorflow.keras.callbacks as callbacks
model_path = 'Xception_New.h5'
model_check_points = callbacks.ModelCheckpoint(filepath= model_path, monitor='val_loss', verbose=0, save_best_only=True,
                                                save_weights_only=False, mode='auto', save_freq='epoch',
                                                options=None)
ES = callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=5, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False )
monitor = 'categorical_accuracy'
# reduce_lr = callbacks.ReduceLROnPlateau(monitor=monitor, patience=3)

In [ ]:
#step_per_epoch = b.shape[0]/8
tf.config.experimental_run_functions_eagerly(True)
tf.config.run_functions_eagerly(True)
history =  model.fit(imgs,b, epochs=15, batch_size = batch_size,
#                      validation_split=0.05,
                     validation_data=(img_val, labs_val),
                    callbacks = [model_check_points])

In [ ]:
imgs,label
#imgs_test,label_test
model.load_weights('Xception_New.h5')
pred_labs = []
for i in range(len(label_test)):
    img =  imgs_test[i,:, :, :]
    img = np.expand_dims(img, axis=0)
    pred = model.predict(img)
    pred = pred.argmax(axis=1)
    pred_labs.append(pred[0])
    
pred_labs=np.array(pred_labs)

In [ ]:
pred_labs = pred_labs+10
pred_labs = np.where(pred_labs==12,0, pred_labs)
pred_labs = np.where(pred_labs==11,2, pred_labs)
pred_labs = np.where(pred_labs==10,1, pred_labs)

label = label_test+10
label = np.where(label==12,0, label)
label = np.where(label==11,2, label)
label = np.where(label==10,1, label)

print('Accuracy is: ', get_accuracy(label, pred_labs))

In [ ]:
array = sklearn.metrics.confusion_matrix(pred_labs, label)
array = array / array.astype(np.float).sum(axis=0)
ax = sn.heatmap(array,annot=True,cmap="YlGnBu", linewidth=0.5,fmt=".1%",xticklabels=['Normal','Benign','Malignant'], yticklabels=['Normal','Benign','Malignant'])

figure = ax.get_figure()
figure.savefig('Xception.png', dpi=1200)